In [1]:
import lsst.sims.maf
lsst.sims.maf.__version__

'2.6.0.sims'

In [2]:
%matplotlib inline   
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# import our python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

## Run minion_1016

In [4]:
# setup database connection
outDir ='LightCurve'
dbFile = '/home/mount/minion_1016_sqlite.db'

# establish connection to sqllite database file
opsimdb = db.OpsimDatabase(dbFile)

# specify output directory for metrics result
resultsDb = db.ResultsDb(outDir=outDir)

In [5]:
from numpy import amin as m

In [6]:
# retrive min inter_night gap, and observation history with the input of database file name and
# arrays of RA and DEC
def find_min_gap(dbFile, ra, dec):
    # While we're in transition between opsim v3 and v4, this may be helpful:
    print("{dbFile} is an opsim version {version} database".format(dbFile=dbFile, version=opsimdb.opsimVersion))
    if opsimdb.opsimVersion == "V3":
        # For v3 databases:
        mjdcol = 'expMJD'
        degrees = False
    else:
        # For v4 and alternate scheduler databases.
        mjdcol = 'observationStartMJD'
        degrees = True
    # Note that astro-lsst-01_2022.db is a v4 database.
    
    # IntraNightGapsMetric returns the gap (in days) between observations within the same night
    # custom reduceFunc to find min gaps 
    metric = metrics.cadenceMetrics.IntraNightGapsMetric(reduceFunc=np.amin, mjdCol=mjdcol)
    # PassMetric just pass all values
    metric_pass = metrics.simpleMetrics.PassMetric(cols=['filter','fiveSigmaDepth', mjdcol, 'expDate'])
    # slicer for slicing pointing history
    slicer = slicers.UserPointsSlicer(ra, dec, lonCol='fieldRA', latCol='fieldDec', latLonDeg=degrees)
    # sql constrains, here I put none
    sql = '' #'night < 365'
    
    # bundles to combine metric, slicer and sql constrain together
    bundle = metricBundles.MetricBundle(metric,slicer,sql)
    date_bundle = metricBundles.MetricBundle(metric_pass, slicer, sql)
    # In case you are using a dither stacker, we can check what columns are actually being pulled from the database.
    print(bundle.dbCols)
    
    # create metric bundle group and returns
    bg = metricBundles.MetricBundleGroup({'gap': bundle, 'date':date_bundle}, opsimdb, outDir=outDir, resultsDb=resultsDb)
    bg.runAll()
    return bg

In [13]:
# provide the ra, dec (must be in degrees) and the sql_lite database dir
ra = 10; dec = 10
b = find_min_gap(dbFile, ra, dec)

/home/mount/minion_1016_sqlite.db is an opsim version V3 database
{'expMJD', 'night', 'fieldRA', 'fieldDec'}
Querying database Summary with no constraint for columns ['expMJD', 'fiveSigmaDepth', 'fieldDec', 'fieldRA', 'filter', 'expDate', 'night'].
Found 2447931 visits
Running:  ['gap', 'date']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [27]:
# assign min intra_night gap in hours to gap
gap = b.bundleDict['gap'].metricValues.data[0]
# assign pointing history to obsHist
obsHist = b.bundleDict['date'].metricValues.data[0]
# put ra, dec, and gap into array 'pos_gap'
pos_gap = [ra, dec, gap]

# Now let's save it to a file, .npz will be automatically added
outfile = '/home/mount/obs_hist'
np.savez(outfile, pos_gap=pos_gap, obsHist=obsHist)